In [15]:
import astropy
from astropy.io import fits 
from astropy.table import Table
from astropy.time import Time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sig

In [16]:
#Goal: define 2 functions to apply to TESS data
#     1. dataClean
#      -remove nan values 
#      -remove flagged values 
#      -correct time values

#     2. flareFinder
#       -compare avg flux value of light curve to flux data
#       -if value_flux > avg flux by x(sigma), pull into new array (until equal to or less than x(sigma)) (new array for each flare?)
#       -then continue through the rest of the data
#       -find max flux in each array 
#       -pull associated timestamp from cleaned data 
#       -print timestamp, flux value for each flare discovered

In [17]:
def dataClean(filename): #Can I use the file path as an input?
    """ This function will remove nan values from TESS 
        20 second lightcurve data and correct time values"""
    
 #Create an array of time and flux data with nans removed    
    with fits.open(filename, mode="readonly") as hdulist:
        raw_time = hdulist[1].data['TIME']
        raw_flux = hdulist[1].data['PDCSAP_FLUX']
        raw_err = hdulist[1].data['PDCSAP_FLUX_ERR']
    
    data = np.vstack((raw_time, raw_flux, raw_err))
    nonan_data = data[:, ~np.isnan(data).any(axis=0)]
 #   error = np.where(np.isfinite(raw_err))
#Apply time correction and create array of corrected and cleaned data
    times = nonan_data[0]
    flux = nonan_data[1]
    error = nonan_data[2]
    t_corr = []
    for i in times:
        r = i + 2457000
        t_corr.append(r)
    time = Time(t_corr, format = 'jd', scale = 'utc')
    time.format = 'iso'
    time = np.array(time)
    flux = np.array(flux)
    err = np.array(error)
#Return cleaned data
    return[time,flux,err]

In [119]:
def flareFinder(cleaned_data, sigma_multiplier):
    """ This function will take cleaned data and a multiple to be applied to sigma. 
        The function will run through the data, creating arrays of flux values 
        when the flux is > avg flux by the specified multiple of sigma. Using the max
        flux value from each array, the corresponding time value will be pulled from
        the cleaned data set. Timestamp and max flux will be printed """
#identify quiescent light curve

    flux_smooth = sig.savgol_filter(cleaned_data[1], 1400, 3) #my opinion of the best parameters for this data
    index = np.where(cleaned_data[1] < (flux_smooth + (3*cleaned_data[2]))) #using 3 as the significance for the error for now
    q_time = cleaned_data[0][index]
    q_flux = sig.savgol_filter(cleaned_data[1][index],2000,3)
    ind2 = np.where(q_flux>0)
    
#find avg flux and flux error from tess data, set as variables, using random number for avg flux
    fluxes = cleaned_data[1]
    times = cleaned_data[0]
    F_err = cleaned_data[2] 
  
 #make sure lengths of data sets are equal 
    flux_list = fluxes[ind2]
    time_list = times[ind2]
    err_list = F_err[ind2]
    
    ix = np.where(flux_list > (q_flux + (sigma_multiplier*err_list)))
    flareFlux = flux_list[ix]
    flareTime = time_list[ix]
    flareTable = pd.DataFrame({
                            'Flux':flareFlux,
                            'Time':flareTime
    })
    
    return[flareTable]

In [120]:
TESSDATA = '/Users/katborski/Documents/TESSGI4/AFPSC/TESS/tess2021232031932-s0042-0000000250081915-0213-a_fast-lc.fits'
cleaned_data = dataClean(TESSDATA)

In [130]:
flareFinder(cleaned_data,3)

[            Flux                     Time
 0    4238.275391  2021-08-21 04:56:35.476
 1    4300.040039  2021-08-21 05:15:15.525
 2    4262.517578  2021-08-21 10:30:16.318
 3    4227.937988  2021-08-21 10:31:36.322
 4    4230.927246  2021-08-21 10:40:36.344
 ..           ...                      ...
 753  4234.636230  2021-09-12 21:52:35.299
 754  4244.537598  2021-09-12 23:26:55.306
 755  4233.805176  2021-09-13 01:08:35.313
 756  4237.577148  2021-09-13 03:00:35.320
 757  4229.994629  2021-09-13 03:20:15.321
 
 [758 rows x 2 columns]]

1